## Mixing Sage and Amber force fields: BRD4 benchmark

This example applies SMIRNOFF-format parameters to BRD4 inhibitors from the [living review on binding free energy benchmark systems](https://www.annualreviews.org/doi/abs/10.1146/annurev-biophys-070816-033654) by Mobley and Gilson. The BRD4 system comes from the [accompanying GitHub repository](https://github.com/MobleyLab/benchmarksets/tree/master/input_files/BRD4).

In [1]:
# Retrieve protein and ligand files for BRD4 and a docked inhibitor from the benchmark systems GitHub repository
# https://github.com/MobleyLab/benchmarksets
import requests

repo_url = (
    "https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/"
)
sources = {
    "receptor.pdb": repo_url + "BRD4/pdb/BRD4.pdb",
    "ligand.pdb": repo_url + "BRD4/pdb/ligand-1.pdb",
    "ligand.sdf": repo_url + "BRD4/sdf/ligand-1.sdf",
}
for filename, url in sources.items():
    r = requests.get(url)
    open(filename, "w").write(r.text)

In [2]:
from openff.toolkit import ForceField, Molecule, Quantity, Topology

Later we will use `Interchange.__add__`, which is an experimental feature. It needs to be turned on by setting the environment variable below, and by doing so we accept [some stability and accuracy risks](https://docs.openforcefield.org/projects/interchange/en/stable/using/experimental.html).

In [3]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [4]:
ligand_molecule = Molecule.from_file("ligand.sdf")
sage = ForceField("openff-2.2.0.offxml")

ligand = sage.create_interchange(topology=ligand_molecule.to_topology())

receptor_topology = Topology.from_pdb("receptor.pdb")

ff14sb = ForceField("ff14sb_off_impropers_0.0.3.offxml")

receptor = ff14sb.create_interchange(topology=receptor_topology)

complex_system = receptor.combine(ligand)

# Pseudo-vacuum
complex_system.box = Quantity([5, 5, 5], "nanometer")


Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



/Users/mattthompson/software/openff-interchange/openff/interchange/operations/_combine.py:52: InterchangeCombinationWarning: Interchange object combination is complex and likely to produce strange results outside of a limited set of use cases it has been tested in. Any workflow using this method is not guaranteed to be suitable for production or stable between versions. Use with extreme caution and thoroughly validate results!
  warnings.warn(


### Export to OpenMM

In [5]:
complex_system.to_openmm()

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x3442730f0> >

### Export to Amber

In [6]:
# TODO: Fix inferring residue information with mixed topology
if False:
    complex_system.to_inpcrd("complex.inpcrd")
    complex_system.to_prmtop("complex.prmtop")

### Export to GROMACS

In [7]:
complex_system.to_gro("complex.gro")
complex_system.to_top("complex.top")